In [14]:
import os, imp, peakutils, datetime, struct, math, sys, time
from Tint_Matlab import *
from ConversionFunctions import find_sub, find_consec, session_datetime, find_basename_files
import numpy as np

import SignalProcessing as sp
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import signal, fftpack
from scipy.interpolate import interp1d
from functools import reduce

import load_intan_rhd_format as load_rhd
from intanutil.notch_filter import notch_filter
from intanutil.read_header import read_header

In [15]:
def write_dummy_pos(position_file, session_parameters):
    
    position_samples = int(session_parameters['duration']*50)

    x = np.zeros((position_samples, 1))
    y = np.zeros_like(x)
    
    with open(position_file, 'wb+') as f:  # opening the .pos file
        trialdate, trialtime = session_datetime(position_file)
        write_list = []
        header_vals = ['trial_data %s' % trialdate,
                  '\r\ntrial_time %s' % trialtime,
                  '\r\nexperimenter %s' % session_parameters['experimenter'],
                  '\r\ncomments Arena:%s' % session_parameters['arena'],
                  '\r\nduration %d' % session_parameters['duration'],
                  '\r\nsw_version %s' % '1.3.0.16',
                  '\r\nnum_colours %d' % 4,
                  '\r\nmin_x %d' % session_parameters['min_x'],
                  '\r\nmax_x %d' % session_parameters['max_x'],
                  '\r\nmin_y %d' % session_parameters['min_y'],
                  '\r\nmax_y %d' % session_parameters['max_y'],
                  '\r\nwindow_min_x %d' % session_parameters['window_min_x'],
                  '\r\nwindow_max_x %d' % session_parameters['window_max_x'],
                  '\r\nwindow_min_y %d' % session_parameters['window_min_y'],
                  '\r\nwindow_max_y %d' % session_parameters['window_max_y'],
                  '\r\ntimebase %d hz' % 50,
                  '\r\nbytes_per_timestamp %d' % 4,
                  '\r\nsample_rate %.1f hz' % 50.0,
                  '\r\nEEG_samples_per_position %d' % 5,
                  '\r\nbearing_colour_1 %d' % 0,
                  '\r\nbearing_colour_2 %d' % 0,
                  '\r\nbearing_colour_3 %d' % 0,
                  '\r\nbearing_colour_4 %d' % 0,
                  '\r\npos_format t,x1,y1,x2,y2,numpix1,numpix2',
                  '\r\nbytes_per_coord %d' % 2,
                  '\r\npixels_per_metre %f' % 100,
                  '\r\nnum_pos_samples %d' % position_samples,
                  '\r\ndata_start']

        for value in header_vals:
            write_list.append(bytes(value, 'utf-8'))

        onespot = 1  # this is just in case we decide to add other modes.

        #write_list = [bytes(headers, 'utf-8')]

        #write_list.append()
        for sample_num in np.arange(0, len(x)):

            '''
            twospot => format: t,x1,y1,x2,y2,numpix1,numpix2
            onespot mode has the same format as two-spot mode except x2 and y2 take on values of 1023 (untracked value)

            note: timestamps and positions are big endian, i has 4 bytes, and h has 2 bytes
            '''

            if onespot == 1:
                numpix1 = 1
                numpix2 = 0
                x2 = 1023
                y2 = 1023
                unused = 0
                total_pix = numpix1  # total number of pixels tracked
                #t_byte = struct.pack('>i', sample_num)
                write_list.append(struct.pack('>i', sample_num))
                #pos_byte = struct.pack('>8h', int(np.rint(main.positions[sample_num,0])), int(np.rint(main.positions[sample_num,1])), x2, y2, numpix1, numpix2, total_pix, unused)

                write_list.append(struct.pack('>8h', int(np.rint(x[sample_num])),
                                              int(np.rint(y[sample_num])), x2, y2, numpix1,
                                              numpix2, total_pix, unused))

        #f.seek(0, 2)
        #f.writelines([bytes(headers, 'utf-8'), t_byte, pos_byte, bytes('\r\ndata_end\r\n', 'utf-8')])
        write_list.append(bytes('\r\ndata_end\r\n', 'utf-8'))
        f.writelines(write_list)

In [16]:
RHD_Directory = 'C:\\Users\\Geoffrey Barrett\\Desktop\\IntanTestData\\03092017'
directory_file_list = os.listdir(RHD_Directory)

rhd_sessions = []  # initializing the list of

intan_basenames = []
# list comprehension that finds the basename of each rhd file and appens to list if it isn't already in the list
[intan_basenames.append(file[:find_sub(file, '_')[-2]]) for file in directory_file_list if '.rhd' in file
 and file[:find_sub(file, '_')[-2]] not in intan_basenames]

for basename in intan_basenames:
    '''iterating through each file with this basename'''
    if basename not in (session_file for session in rhd_sessions for session_file in session):
        basename_session = find_basename_files(basename, RHD_Directory)

    if basename_session not in (session_file for session in rhd_sessions for session_file in session):
        for session in basename_session:
            rhd_sessions.append(session)


In [17]:
print(rhd_sessions[2])

['C:\\Users\\Geoffrey Barrett\\Desktop\\IntanTestData\\03092017\\cage_test_electronics_170309_123009.rhd', 'C:\\Users\\Geoffrey Barrett\\Desktop\\IntanTestData\\03092017\\cage_test_electronics_170309_122809.rhd', 'C:\\Users\\Geoffrey Barrett\\Desktop\\IntanTestData\\03092017\\cage_test_electronics_170309_122609.rhd', 'C:\\Users\\Geoffrey Barrett\\Desktop\\IntanTestData\\03092017\\cage_test_electronics_170309_122409.rhd', 'C:\\Users\\Geoffrey Barrett\\Desktop\\IntanTestData\\03092017\\cage_test_electronics_170309_122209.rhd']


In [18]:
for session in rhd_sessions:
    
    tint_basename = os.path.basename(os.path.splitext(sorted(session, reverse=False)[0])[0]) 
    # the first created file of the session will be the basename for tint
    print('Loading the Intan files associated with the following basename: %s!' % tint_basename)
    session_position_file = '%s.pos' % os.path.join(RHD_Directory, tint_basename)
    
    data = np.array([])
    t_intan = np.array([])
    # concatenates the data from all the .rhd files
    for session_file in sorted(session, reverse=False):
        
        print('Currently loading data from the following file: %s' % session_file)
        file_data = load_rhd.read_data(session_file) # loading the .rhd data
        
        #file_header = read_header(open(session_file, 'rb'))  # if we need the header information
        
        # Acquiring session information 
        session_info = file_data['amplifier_channels']  # information about the channels (port_name, electrical impedance, etc)
        
        if data.shape[0] == 0:
            data = file_data['amplifier_data']
            #bits, data is arranged into (number of channels, number of samples)
        else:
            
            data = np.concatenate((data,file_data['amplifier_data']), axis=1)
            #print(data.shape)
        
        if t_intan.shape[0] == 0:
            frequency_parameters = file_data['frequency_parameters']  # the frequency settings of the intan recording system
            Fs_intan = frequency_parameters['amplifier_sample_rate']  # the sampling frequency of the intan recording system
            t_intan =  file_data['t_amplifier'] # the times recorded by the intan system, starts at the value of 0 seconds
        else:
            # the time's always start at 0, per .rhd file, so when putting them together you need add to each time value
            t_intan = np.concatenate((t_intan,file_data['t_amplifier']), axis=0)   
            #print(t_intan.shape)
    
    file_data = None
    
    duration = len(t_intan)/Fs_intan # the duration value that would be reported in the .pos/.set settings
    #print(duration)
    if not duration.is_integer():
        '''Tint needs integer values in the duration spot of the .pos file, so we will round down if not an integer'''
        duration = math.floor(duration) # the new integer duration value

    # dumby position parameters
    experimenter_name = 'Geoff'
    arena = 'Circular Track'
    min_x = 0  # found in Tint's field view
    max_x = 512  # found in Tint's field view
    min_y = 0  # found in Tint's field view
    max_y = 523  # found in Tint's field view

    window_min_x = 200  # found in Tint's field view
    window_max_x = 538  # found in Tint's field view
    window_min_y = 164  # found in Tint's field view
    window_max_y = 492  # found in Tint's field view

    session_parameters = {'experimenter':experimenter_name, 'arena': arena, 'duration': duration, 'min_x':min_x, 'max_x':max_x,
                         'min_y':min_y, 'max_y':max_y, 'window_min_x':window_min_x, 'window_max_x':window_max_x,
                         'window_min_y':window_min_y, 'window_max_y':window_max_y, 'ppm':100}
    
    print('Writing the dumby position file!')
    write_dummy_pos(session_position_file, session_parameters)
print('Finished')

Loading the Intan files associated with the following basename: cage_test_170309_121452!
Currently loading data from the following file: C:\Users\Geoffrey Barrett\Desktop\IntanTestData\03092017\cage_test_170309_121452.rhd
Currently loading data from the following file: C:\Users\Geoffrey Barrett\Desktop\IntanTestData\03092017\cage_test_170309_121652.rhd
Currently loading data from the following file: C:\Users\Geoffrey Barrett\Desktop\IntanTestData\03092017\cage_test_170309_121852.rhd
Writing the dumby position file!
Loading the Intan files associated with the following basename: cage_test_170309_121033!
Currently loading data from the following file: C:\Users\Geoffrey Barrett\Desktop\IntanTestData\03092017\cage_test_170309_121033.rhd
Currently loading data from the following file: C:\Users\Geoffrey Barrett\Desktop\IntanTestData\03092017\cage_test_170309_121133.rhd
Currently loading data from the following file: C:\Users\Geoffrey Barrett\Desktop\IntanTestData\03092017\cage_test_170309_12